# A Meeting Analysis Use Case

Copyright 2025-2026, Denis Rothman

Mục tiêu của sổ tay này là minh họa cách hướng dẫn một mô hình AI thực hiện quy trình phân tích nhiều bước, từ bản ghi thô đến những thông tin có thể hành động được, qua đó đào tạo cả người dùng lẫn AI.

Sổ tay này là một hướng dẫn từng bước về cách bạn xây dựng một ngữ cảnh có cấu trúc. Hướng dẫn này sẽ đóng vai trò như bản thiết kế kiến trúc cho mã nguồn của bạn, giải thích không chỉ điều gì cần làm mà còn lý do vì sao cần làm điều đó ở mỗi giai đoạn.

![](Use_case_flowchart.png)

In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

True

In [6]:
meeting_transcript = """
Tom: Chào buổi sáng cả nhà. Cafe vẫn chưa đủ đô để giúp tôi tỉnh táo hẳn nữa.
Sarah: Chào Tom. Được rồi, chúng ta bắt đầu thôi nhỉ. Về tiến độ Dự án Phoenix. Tom này, anh nói là các thành phần backend vẫn đang đi đúng hướng chứ?
Tom: Cơ bản là vậy. Bọn tôi có gặp một chút trục trặc với việc tích hợp cổng thanh toán. Nó... phức tạp hơn những gì tài liệu mô tả. Chắc là chúng tôi cần thêm khoảng ba ngày nữa.
Maria: Ba ngày á? Tom ơi, như vậy là đẩy giai đoạn kiểm thử cuối cùng sát nút ngày ra mắt rồi. Chúng ta không có khoảng thời gian dự phòng dư dả thế đâu.
Sarah: Tôi đồng ý với Maria. Có phương án nào khác không Tom?
Tom: Tôi nghĩ chắc mình phải làm bù vào cuối tuần để kịp tiến độ thôi. Thú thực là tôi không muốn đâu, nhưng tôi hiểu tình thế khó khăn hiện tại của chúng ta.
Sarah: Rất ghi nhận nỗ lực của anh, Tom. Vậy chúng ta cứ tạm thống nhất như thế nhé. Maria, còn phần front-end thì sao rồi?
Maria: Bên tôi ổn. Thực tế là đang vượt tiến độ một chút, nên chúng tôi cũng có dư chút nguồn lực.
Sarah: Tuyệt vời. Được rồi, một việc cuối cùng nhé. Đội Marketing muốn đẩy mạnh truyền thông trên mạng xã hội vào đúng ngày ra mắt. Mọi người thấy sao?
Tom: Nghe cũng hợp lý, theo đúng quy chuẩn thôi.
Maria: Tôi lại nghĩ đó là một sai lầm. Đẩy mạnh ngay ngày đầu sẽ làm máy chủ bị quá tải nếu chẳng may còn lỗi tồn đọng. Thay vào đó, mình nên "soft launch" (ra mắt thử nghiệm), chỉ giới hạn người tham gia trong tuần đầu tiên, sau đó mới bùng nổ truyền thông. Như vậy sẽ dễ kiểm soát hơn.
Sarah: Ý kiến rất hay đấy Maria. Chiến lược này an toàn hơn nhiều. Quyết định vậy nhé. Được rồi, buổi họp rất hiệu quả. Tôi sẽ gửi lại bản tóm tắt cho mọi người.
Tom: Nhất trí. Giờ thì đi nạp thêm cafe thôi.
"""

-----

### Layer 1: Establishing the scope (The 'What')

Ở đây, chúng ta xác định phạm vi của phân tích. Đầu ra của mỗi bước sẽ làm cơ sở cho bước tiếp theo, tạo thành một chuỗi ngữ cảnh.

1.  **Cell 4: g2 - Set the Signal-to-Noise Ratio**

Chúng ta sẽ bắt đầu bằng việc làm sạch dữ liệu. Yêu cầu rõ ràng yêu cầu AI tách nội dung có tính chất thực chất ra khỏi conversational noise.

In [8]:
prompt_g2 = f"""
        Bạn là một chuyên gia phân tích dữ liệu hội thoại. Hãy phân tích bản ghi chép (Transcript) cuộc họp dưới đây. 
        Nhiệm vụ của bạn là tách biệt "Substantive content" (nội dung trọng tâm) khỏi "Conversational noise" (nhiễu hội thoại).

        - Substantive content bao gồm: các quyết định (decisions), cập nhật dự án (project updates), các vấn đề phát sinh (problems raised), và đề xuất chiến lược (strategic suggestions).
        - Noise bao gồm: lời chào hỏi (greetings), xã giao (pleasantries), và các trao đổi ngoài lề không liên quan đến công việc (off-topic remarks).

        Yêu cầu:
        1. CHỈ trả về "Substantive content".
        2. Loại bỏ hoàn toàn mọi lời dẫn giải hoặc kết luận của AI.
        3. Trình bày dưới dạng danh sách các ý chính (bullet points) súc tích.

        Transcript:
        ---
        {meeting_transcript}
        ---
        """

In [9]:
from openai import OpenAI

try:
    client = OpenAI()

    response_g2 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[
            {"role": "user", "content": prompt_g2}
        ]
    )

    substantive_content = response_g2.choices[0].message.content
    print("--- SUBSTANTIVE CONTENT ---")
    print(substantive_content)

except Exception as e:
    print(f"An error occurred: {e}")

--- SUBSTANTIVE CONTENT ---
- Backend Dự án Phoenix: gặp trục trặc khi tích hợp cổng thanh toán, phức tạp hơn tài liệu mô tả; cần thêm khoảng 3 ngày để hoàn thành.
- Rủi ro: chậm 3 ngày sẽ đẩy giai đoạn kiểm thử cuối cùng sát ngày ra mắt, thiếu thời gian dự phòng.
- Phương án xử lý: Tom sẽ làm bù cuối tuần để kịp tiến độ; tạm thống nhất theo phương án này.
- Front-end: đang ổn và vượt tiến độ nhẹ; có dư nguồn lực.
- Marketing: đề xuất đẩy mạnh truyền thông mạng xã hội đúng ngày ra mắt.
- Vấn đề được nêu: đẩy mạnh truyền thông ngày đầu có thể gây quá tải máy chủ nếu còn lỗi tồn đọng.
- Đề xuất chiến lược: “soft launch” tuần đầu (giới hạn người tham gia), sau đó mới tăng cường truyền thông để dễ kiểm soát.
- Quyết định: chọn chiến lược “soft launch” thay vì bùng nổ truyền thông ngay ngày ra mắt.


2.  **Cell 5: g3 - Define the Scope of Time (Simulated RAG)**

Chúng ta sẽ mô phỏng bối cảnh RAG bằng cách cung cấp một bản tóm tắt 'trước đó' và yêu cầu xác định những nội dung mới. Việc này giúp người dùng hiểu được tầm quan trọng của bối cảnh lịch sử.

In [11]:
previous_summary = "Trong cuộc họp trước, chúng ta đã thống nhất các mục tiêu cho Dự án Phoenix, đồng thời phân công phần việc backend cho Tom và front-end cho Maria."
previous_summary

'Trong cuộc họp trước, chúng ta đã thống nhất các mục tiêu cho Dự án Phoenix, đồng thời phân công phần việc backend cho Tom và front-end cho Maria.'

In [12]:
prompt_g3 = f"""
Context: Tóm tắt của cuộc họp trước là: "{previous_summary}"

Task: Phân tích nội dung trọng tâm (substantive content) từ cuộc họp mới dưới đây. Hãy đối chiếu và CHỈ xác định, tóm tắt những diễn biến mới, các vấn đề phát sinh, hoặc các quyết định vừa được đưa ra kể từ cuộc họp trước. Tuyệt đối không lặp lại các thông tin đã có trong bản tóm tắt cũ.

New Meeting Content:
---
{substantive_content}
---
"""

In [13]:
try:
    response_g3 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g3}]
    )
    new_developments = response_g3.choices[0].message.content
    print("--- NEW DEVELOPMENTS SINCE LAST MEETING ---")
    print(new_developments)
except Exception as e:
    print(f"An error occurred: {e}")

--- NEW DEVELOPMENTS SINCE LAST MEETING ---
- **Phát sinh ở backend (tích hợp cổng thanh toán):** Việc tích hợp gặp trục trặc và **phức tạp hơn so với tài liệu**, dẫn đến **ước tính cần thêm ~3 ngày** để hoàn thành.

- **Rủi ro tiến độ mới:** Nếu trễ 3 ngày sẽ **ép giai đoạn kiểm thử cuối** sát ngày ra mắt, **giảm/thiếu thời gian dự phòng**.

- **Phương án xử lý tạm thời:** **Tom sẽ làm bù cuối tuần** để bù lại phần chậm và **tạm thống nhất** đi theo phương án này.

- **Tình trạng front-end:** Front-end **đang ổn và hơi vượt tiến độ**, vì vậy **có dư nguồn lực** (hàm ý có thể hỗ trợ nơi khác nếu cần).

- **Vấn đề/đánh giá mới về marketing & vận hành:** Có đề xuất **đẩy mạnh truyền thông mạng xã hội đúng ngày ra mắt**, nhưng nêu lo ngại rằng làm vậy **có thể gây quá tải máy chủ** nếu hệ thống vẫn còn lỗi tồn đọng.

- **Quyết định chiến lược ra mắt:** **Chọn “soft launch” trong tuần đầu (giới hạn người tham gia)**, sau đó mới tăng cường truyền thông; **không chọn phương án bùng nổ truyền

### Layer 2: Conducting the Investigation (The 'How')

Bây giờ chúng ta chuyển từ việc xác định các sự kiện sang việc tạo ra những hiểu biết sâu sắc, đây là cốt lõi của hành trình diễn giải ngữ nghĩa trong bối cảnh.

1.  **Cell 6: g4 - Identify the Key Threads**

Đây là một bước quan trọng. Câu lệnh yêu cầu AI đọc giữa những dòng.

In [14]:
prompt_g4 = f"""
Task: Phân tích nội dung cuộc họp dưới đây để xác định các implicit social dynamics (động lực xã hội ngầm) và các cảm xúc chưa được bộc lộ rõ ràng (unstated feelings). Hãy phân tích sâu hơn ý nghĩa bề mặt của câu chữ (literal words).

- Có ai biểu hiện sự ngập ngừng (hesitant) hoặc miễn cưỡng (reluctant) dù bên ngoài vẫn đồng ý với một vấn đề nào đó không?
- Có sự tồn tại của bất kỳ sự bất đồng hoặc căng thẳng ngầm (underlying disagreements/tensions) nào không?
- Tâm trạng và bầu không khí chung (overall mood) của buổi họp là gì?

Meeting Content:
---
{substantive_content}
---
"""

In [15]:
try:
    response_g4 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g4}]
    )
    implicit_threads = response_g4.choices[0].message.content
    print("--- IMPLICIT THREADS AND DYNAMICS ---")
    print(implicit_threads)
except Exception as e:
    print(f"An error occurred: {e}")

--- IMPLICIT THREADS AND DYNAMICS ---
## 1) Có ai biểu hiện ngập ngừng/miễn cưỡng dù bên ngoài vẫn đồng ý?

### Tom (Backend) – dấu hiệu “đồng ý nhưng gồng”
- **Bề mặt:** “Tom sẽ làm bù cuối tuần để kịp tiến độ; tạm thống nhất theo phương án này.”
- **Hàm ý xã hội:** Cụm *“làm bù cuối tuần”* thường là một **cam kết mang tính “hy sinh cá nhân”** để bảo vệ deadline, hay để giảm áp lực lên kế hoạch chung. Điều này dễ xuất hiện khi người nói **không thật sự có lựa chọn tốt** (hoặc không muốn bị xem là “nguyên nhân làm trễ”).
- **Cảm xúc chưa nói:** khả năng có **stress, lo lắng bị đánh giá năng lực**, và/hoặc **bực bội vì tài liệu/đối tác thanh toán không rõ ràng**. Việc “nhận làm thêm” đôi khi là cách **dập căng thẳng ngay tại cuộc họp** thay vì tranh luận về nguyên nhân gốc hoặc đòi hỏi thay đổi kế hoạch.

### Cả nhóm – dấu hiệu “tạm đồng thuận”
- **Bề mặt:** “tạm thống nhất theo phương án này.”
- **Hàm ý:** từ “tạm” là tín hiệu điển hình của **sự miễn cưỡng tập thể**: đồng ý để đi tiếp,

2.  **Cell 7: g5 - Perform "Intellectual Combinations"**

Ở đây, chúng tôi yêu cầu AI thể hiện tính sáng tạo và giải quyết một vấn đề bằng cách tổng hợp các ý tưởng khác nhau từ cuộc họp.

In [16]:
prompt_g5 = f"""
Context: Trong cuộc họp, Maria đã đề xuất một chiến lược 'soft launch' để tránh gây áp lực cho hệ thống (server strain), đồng thời cho biết đội của cô ấy hiện đang có 'extra bandwidth'. Trong khi đó, Tom đang đối mặt với tình trạng chậm trễ 3 ngày (3-day delay) ở phần backend.

Task: Hãy đề xuất một ý tưởng mới lạ (novel) và có tính thực thi cao (actionable) bằng cách tận dụng 'extra bandwidth' từ đội của Maria để giảm thiểu (mitigate) tình trạng '3-day delay' của Tom. Hãy kết hợp hai thông tin riêng biệt này thành một giải pháp đồng nhất (single solution).
"""

try:
    response_g5 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g5}]
    )
    novel_solution = response_g5.choices[0].message.content
    print("--- NOVEL SOLUTION PROPOSED BY AI ---")
    print(novel_solution)
except Exception as e:
    print(f"An error occurred: {e}")

--- NOVEL SOLUTION PROPOSED BY AI ---
Một giải pháp “một mũi tên trúng hai đích” là triển khai **Soft Launch kiểu “Shadow Backend + Feature Flag”**: đội Maria dùng *extra bandwidth* để dựng một lớp backend “song song” (shadow) cho phần Tom đang bị trễ, rồi đưa vào **soft launch** để chạy thử với một nhóm nhỏ traffic. Cách này vừa giảm tải rủi ro server strain, vừa kéo tiến độ backend về gần “đúng hạn” bằng cách tách phần có thể làm ngay ra khỏi phần đang bị nghẽn.

## Ý tưởng cốt lõi
**Maria’s team xây một “shadow path”**: một phiên bản tối thiểu nhưng chạy được của các endpoint/logic đang bị trễ (có thể là read-only hoặc degraded), đặt sau **feature flag** và **traffic splitting** (ví dụ 1–5% người dùng).  
Tom tiếp tục hoàn thiện “gold path” backend chuẩn. Khi Tom xong, chỉ cần **switch flag**/tăng dần traffic sang gold path.

## Vì sao giúp giảm “3-day delay”
- **Gỡ nút thắt**: Maria làm phần có thể tách ra (API contract, stub service, cache layer, read replica, async jobs, fallback

-----

### Layer 3: Determining the Action (The 'What Next')

Cuối cùng, chúng ta chuyển phân tích thành các sản phẩm cụ thể, hướng tới tương lai.

1.  **Cell 8: g6 - Define the Output Format (Final Summary)**

      * Chúng tôi tổng hợp tất cả thông tin quan trọng thành một bản tóm tắt cuối cùng có cấu trúc, thể hiện tầm quan trọng của kết quả rõ ràng.


In [17]:
prompt_g6 = f"""
Task: Tạo một bản tóm tắt (summary) cuối cùng và súc tích (concise) về cuộc họp dưới dạng một markdown table. 
Hãy sử dụng thông tin sau để xây dựng bảng:

- New Developments: {new_developments}

Bảng (table) phải có đúng ba cột với tiêu đề chính xác như sau: "Topic", "Decision/Outcome", và "Owner".
"""

try:
    response_g6 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g6}]
    )
    final_summary_table = response_g6.choices[0].message.content
    print("--- FINAL MEETING SUMMARY TABLE ---")
    print(final_summary_table)
except Exception as e:
    print(f"An error occurred: {e}")

--- FINAL MEETING SUMMARY TABLE ---
| Topic | Decision/Outcome | Owner |
|---|---|---|
| Backend – tích hợp cổng thanh toán | Tích hợp phức tạp hơn tài liệu, phát sinh trục trặc; ước tính cần thêm ~3 ngày để hoàn thành | Backend team |
| Rủi ro tiến độ | Nếu trễ 3 ngày sẽ ép kiểm thử cuối sát ngày ra mắt, giảm/thiếu buffer | PM/Project team |
| Phương án xử lý tạm thời | Tom làm bù cuối tuần để bù tiến độ; tạm thời thống nhất theo phương án này | Tom |
| Front-end status | Front-end ổn và hơi vượt tiến độ; có dư nguồn lực để hỗ trợ nếu cần | Front-end team |
| Marketing & vận hành – rủi ro tải hệ thống | Lo ngại đẩy mạnh truyền thông đúng ngày ra mắt có thể gây quá tải nếu còn lỗi tồn đọng | Marketing + Ops |
| Chiến lược ra mắt | Chọn “soft launch” tuần đầu (giới hạn người tham gia), sau đó mới tăng truyền thông; không bùng nổ truyền thông ngay ngày ra mắt | PM + Marketing |


| Topic | Decision/Outcome | Owner |
|---|---|---|
| Backend – tích hợp cổng thanh toán | Tích hợp phức tạp hơn tài liệu, phát sinh trục trặc; ước tính cần thêm ~3 ngày để hoàn thành | Backend team |
| Rủi ro tiến độ | Nếu trễ 3 ngày sẽ ép kiểm thử cuối sát ngày ra mắt, giảm/thiếu buffer | PM/Project team |
| Phương án xử lý tạm thời | Tom làm bù cuối tuần để bù tiến độ; tạm thời thống nhất theo phương án này | Tom |
| Front-end status | Front-end ổn và hơi vượt tiến độ; có dư nguồn lực để hỗ trợ nếu cần | Front-end team |
| Marketing & vận hành – rủi ro tải hệ thống | Lo ngại đẩy mạnh truyền thông đúng ngày ra mắt có thể gây quá tải nếu còn lỗi tồn đọng | Marketing + Ops |
| Chiến lược ra mắt | Chọn “soft launch” tuần đầu (giới hạn người tham gia), sau đó mới tăng truyền thông; không bùng nổ truyền thông ngay ngày ra mắt | PM + Marketing |

2.  **Cell 9: g7 - Generate the Subsequent Task**

      * Bước cuối cùng là sử dụng phân tích để thực hiện một hành động trong thế giới thực, khép kín vòng phản hồi từ nhận thức đến hành động.

In [18]:
prompt_g7 = f"""
Task: Dựa trên Summary Table dưới đây, hãy soạn thảo một follow-up email với văn phong lịch sự (polite) và chuyên nghiệp (professional) để gửi cho nhóm (Sarah, Tom, Maria). 
Email cần trình bày rõ ràng các quyết định đã được thông qua (decisions made) và các hạng mục công việc (action items) cụ thể cho từng người.

Summary Table:
---
{final_summary_table}
---
"""

try:
    response_g7 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g7}]
    )
    follow_up_email = response_g7.choices[0].message.content
    print("--- DRAFT FOLLOW-UP EMAIL ---")
    print(follow_up_email)
except Exception as e:
    print(f"An error occurred: {e}")

--- DRAFT FOLLOW-UP EMAIL ---
Subject: Follow-up: Decisions & Action Items – Payment Integration, Schedule Risk, and Launch Strategy

Hi Sarah, Tom, Maria,  

Cảm ơn mọi người đã trao đổi. Dưới đây là tóm tắt các **quyết định đã thống nhất** và **action items** tương ứng để cả nhóm cùng bám sát kế hoạch.

## 1) Decisions made (đã thông qua)
1. **Backend – tích hợp cổng thanh toán:** Việc tích hợp phức tạp hơn so với tài liệu và đang phát sinh trục trặc; ước tính cần **thêm ~3 ngày** để hoàn tất.  
2. **Rủi ro tiến độ:** Nếu trễ ~3 ngày sẽ dồn khối lượng **final testing** sát ngày ra mắt và **giảm/thiếu buffer**.  
3. **Phương án xử lý tạm thời:** Thống nhất **Tom làm bù cuối tuần** để bù tiến độ (áp dụng như phương án tạm thời hiện tại).  
4. **Front-end status:** Front-end đang ổn và hơi vượt tiến độ; có thể **hỗ trợ thêm nguồn lực** nếu cần.  
5. **Chiến lược ra mắt:** Chọn **soft launch trong tuần đầu** (giới hạn người tham gia), sau đó mới tăng truyền thông; **không đẩy mạnh truyền